In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib widget
from aiida import load_profile
profile = load_profile()

import os
import numpy as np
import math as m
import pymatgen
import scipy.constants as cons
import matplotlib.pyplot as plt
from aiida.orm import load_node, Code, load_group, Group, WorkChainNode, Dict, StructureData
from aiida.engine import submit
from aiida_kkr.tools import find_parent_structure, plot_kkr, kkrparams, neworder_potential_wf
from aiida_kkr.calculations import KkrCalculation, VoronoiCalculation, KkrimpCalculation
from aiida_kkr.workflows import kkr_dos_wc, kkr_bs_wc, kkr_scf_wc, kkr_imp_sub_wc

In [2]:
import matplotlib
scf_lly = load_node('23b13d9d-6e02-46c7-b3ac-17033b205b19')
struc0 = find_parent_structure(scf_lly)
### Form here Checking for structure
dis_set= set()
dis_to_site_map= {}
for site in struc0.sites:
    pos= site.position
    dis = np.sqrt(np.sum([i**2 for i in pos]))
    dis_set.add(dis)
    dis_to_site_map[dis] = site
    
alat= 8.2826* 0.529#*10**(-10) Angstom
relative= 1
b_hor= 0.529*10**(-10)

for dis in dis_set:
    print(f"Site : {[ i*relative for i in dis_to_site_map[dis].position]}, kind name : {dis_to_site_map[dis].kind_name}, distance: {dis}")
struc0.attributes

BRAVAIS= [[-0.5, -0.2886751345, 2.3185997682],
  [0.5, -0.2886751345, 2.3185997682],
  [0.0, 0.577350269, 2.3185997682]],

BRAVAIS_arr= np.array([[-2.1907477 , -1.26482877, 10.15893422],
        [ 2.1907477 , -1.26482877, 10.15893422],
        [ 0.        ,  2.52965755, 10.15893422]])# Angstom



scf_lly.inputs.parameters.get_dict()
atom_list= [[0.0, 0.0, 0.0],
  [0.212, 0.212, 0.212],
  [-0.212, -0.212, -0.212],
  [0.4, 0.4, 0.4],
  [-0.4, -0.4, -0.4],
  [0.1, 0.1, 0.1],
  [-0.1, -0.1, -0.1],
  [0.3, 0.3, 0.3],
  [-0.3, -0.3, -0.3],
  [0.5, 0.5, 0.5]]
atom_arr= np.array(atom_list)*alat
atom_arr
calculated_site_list= []

for atom_pos in atom_list:
    calculated_site=0
#    print(f"atom_pos : {atom_pos}")
    i=0
    for cor in atom_pos:

        index = i%3
        i += 1
 
 #   print(BRAVAIS_arr[0,index,:]*cor) 

#        print(F"Index : {index},  BRAVAIS_arr : {BRAVAIS_arr[index,:]}")
#        calculated_site += BRAVAIS_arr[index,:]*cor
 #       print(calculated_site)
              
    calculated_site_list.append(calculated_site)
    
#calculated_site_list
atom_arr
alat
struc0.cell

Site : [0.0, 0.0, 0.0], kind name : Te, distance: 0.0
Site : [-3.8809943008617e-17, 1.0071414359274e-16, 21.3409084951], kind name : HX, distance: 21.3409084951
Site : [2.1914815820078, 1.2652524812963, 22.357142232962], kind name : Bi, distance: 22.499894759544965
Site : [1.8747016921119e-16, 2.5305049625927, 23.373375970824], kind name : HX, distance: 23.509958733251725
Site : [-7.5181394577443e-17, -2.3960630364491e-17, 24.023765563056], kind name : Te, distance: 24.023765563056
Site : [2.1914815820078, 1.2652524812963, 25.405843446548], kind name : HX, distance: 25.531555702624388
Site : [-2.0670559458015e-16, 2.5305049625927, 26.78792133004], kind name : Te, distance: 26.90717719401495
Site : [-4.6526674632394e-17, 1.2427329655867e-16, 27.438310922272], kind name : HX, distance: 27.438310922272
Site : [2.1914815820078, 1.2652524812963, 28.454544660134], kind name : Bi, distance: 28.566843843541175
Site : [1.7975343758741e-16, 2.5305049625927, 29.470778397996], kind name : HX, dist

[[2.1914815820078, 1.2652524812963, 10.162337378619],
 [-2.1914815820078, 1.2652524812963, 10.162337378619],
 [-8.1101110346236e-17, -2.5305049625927, 10.162337378619]]

In [13]:
# To check the atomic shell in around the cluster center atom.
from pathlib import Path
import numpy as np

#file = Path.open('./clusters')

with open('./clusters', 'r') as f:
    lines= f.readlines()  
    collecte_data = False
    
    
    separation_line= ['1       1',
                      '2       2',
                      '3       3',
                      '4       4',
                      '5       5',
                      '6       6',
                      '7       7',
                      '8       8',
                      '9       9',
                      '10       10',]
    all_cluster_shell=[]
    all_cluster_shell_data=[]
    for para_ind, para in enumerate(separation_line[0:5]):
        collecte_data= False
        skip= []
        collected_line = []
        for ind, line in enumerate(lines):
            if collecte_data:
                if ind in skip:
                    continue
                data_list = [float(i.replace('\n','')) for i in line.split(' ') if i != '']
                collected_line.append(data_list)

            if separation_line[para_ind] in line:
                collecte_data = True
                num= skip.append(ind+2)
                
            if separation_line[para_ind + 1] in lines[ind+2]:
                break

            if separation_line[para_ind + 1] in lines[ind+1]:
                break
        data_arr = np.array(collected_line)
        #print(data_arr)
        data = []
        shell_dis = []
        for i in range(len(data_arr)):
            if data_arr[i,-2] != 0.0:
                data.append(list(data_arr[i,:]))
                if data_arr[i,-1] not in shell_dis:
                    shell_dis.append(data_arr[i,-1])
        all_cluster_shell_data.append(data)
        all_cluster_shell.append(shell_dis)

In [14]:
all_cluster_shell

[[0.0,
  0.740519769,
  1.0,
  1.02255524,
  1.24433497,
  1.43025146,
  1.47462945,
  1.59636134],
 [0.0,
  0.691320238,
  0.85502297,
  1.0,
  1.02255524,
  1.16416122,
  1.21569884,
  1.30769027,
  1.31569916,
  1.43025146,
  1.47462945,
  1.53468933,
  1.5741422],
 [0.0,
  0.691320238,
  0.85502297,
  1.0,
  1.02255524,
  1.16416122,
  1.21569884,
  1.30769027,
  1.31569916,
  1.43025146,
  1.47462945,
  1.53468933,
  1.5741422],
 [0.0,
  0.691320238,
  0.740519769,
  1.0,
  1.09246424,
  1.16416122,
  1.21569884,
  1.24433497,
  1.30769027,
  1.39115986,
  1.48103954,
  1.53468933,
  1.5741422,
  1.59636134],
 [0.0,
  0.691320238,
  0.740519769,
  1.0,
  1.09246424,
  1.16416122,
  1.21569884,
  1.24433497,
  1.30769027,
  1.39115986,
  1.48103954,
  1.53468933,
  1.5741422,
  1.59636134]]

In [4]:
alat
0.740519769*alat, 1.0*alat, 1.02255524*alat, 1.24433497*alat, 1.43025146*alat, 1.59636134*alat

(3.244583961482563,
 4.3814954,
 4.480321080305896,
 5.4520479471141385,
 6.266640192833285,
 6.994449867947837)

In [5]:
plot_kkr(struc0, silent= True, stattic= True, )

structure has vacancies, need to remove empty sites for plotting
removing atom kind name 'HX' @ 1.8747016921119e-16,2.5305049625927,23.373375970824
removing atom kind name 'HX' @ -4.6526674632394e-17,1.2427329655867e-16,27.438310922272
removing atom kind name 'HX' @ 1.7975343758741e-16,2.5305049625927,29.470778397996
removing atom kind name 'HX' @ -3.8809943008617e-17,1.0071414359274e-16,21.3409084951
removing atom kind name 'HX' @ 2.1914815820078,1.2652524812963,25.405843446548


ValueError: top_level must have an `_ipython_display_` method

In [6]:
struc0.get_pymatgen()

Structure Summary
Lattice
    abc : 10.472657559692419 10.472657559692419 10.472657559692438
 angles : 24.157680577616187 24.157680577616187 24.157680577615878
 volume : 169.06740315520517
      A : 2.1914815820078 1.2652524812963 10.162337378619
      B : -2.1914815820078 1.2652524812963 10.162337378619
      C : -8.1101110346236e-17 -2.5305049625927 10.162337378619
PeriodicSite: Te (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Te (-0.0000, 2.5305, 26.7879) [1.2120, 1.2120, 0.2120]
PeriodicSite: Te (-0.0000, 0.0000, 24.0238) [0.7880, 0.7880, 0.7880]
PeriodicSite: Bi (2.1915, 1.2653, 22.3571) [1.4000, 0.4000, 0.4000]
PeriodicSite: Bi (2.1915, 1.2653, 28.4545) [1.6000, 0.6000, 0.6000]
PeriodicSite:  (0.0000, 2.5305, 23.3734) [1.1000, 1.1000, 0.1000]
PeriodicSite:  (-0.0000, 0.0000, 27.4383) [0.9000, 0.9000, 0.9000]
PeriodicSite:  (-0.0000, 2.5305, 29.4708) [1.3000, 1.3000, 0.3000]
PeriodicSite:  (-0.0000, -0.0000, 21.3409) [0.7000, 0.7000, 0.7000]
PeriodicSite:  (2.191

In [7]:
BRAVAIS_arr = np.array(struc0.cell)
BRAVAIS_arr_new = BRAVAIS_arr.copy()
i = 0
for j in range(len(BRAVAIS_arr)):
    if j<=1:
        BRAVAIS_arr_new[j,:] = BRAVAIS_arr_new[j,:] - BRAVAIS_arr[2, :]

BRAVAIS_arr_new

array([[ 2.19148158e+00,  3.79575744e+00,  0.00000000e+00],
       [-2.19148158e+00,  3.79575744e+00,  0.00000000e+00],
       [-8.11011103e-17, -2.53050496e+00,  1.01623374e+01]])

In [8]:
pymatget = struc0.get_pymatgen_structure()
sites = pymatget.sites

pymatget

Structure Summary
Lattice
    abc : 10.472657559692419 10.472657559692419 10.472657559692438
 angles : 24.157680577616187 24.157680577616187 24.157680577615878
 volume : 169.06740315520517
      A : 2.1914815820078 1.2652524812963 10.162337378619
      B : -2.1914815820078 1.2652524812963 10.162337378619
      C : -8.1101110346236e-17 -2.5305049625927 10.162337378619
PeriodicSite: Te (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Te (-0.0000, 2.5305, 26.7879) [1.2120, 1.2120, 0.2120]
PeriodicSite: Te (-0.0000, 0.0000, 24.0238) [0.7880, 0.7880, 0.7880]
PeriodicSite: Bi (2.1915, 1.2653, 22.3571) [1.4000, 0.4000, 0.4000]
PeriodicSite: Bi (2.1915, 1.2653, 28.4545) [1.6000, 0.6000, 0.6000]
PeriodicSite:  (0.0000, 2.5305, 23.3734) [1.1000, 1.1000, 0.1000]
PeriodicSite:  (-0.0000, 0.0000, 27.4383) [0.9000, 0.9000, 0.9000]
PeriodicSite:  (-0.0000, 2.5305, 29.4708) [1.3000, 1.3000, 0.3000]
PeriodicSite:  (-0.0000, -0.0000, 21.3409) [0.7000, 0.7000, 0.7000]
PeriodicSite:  (2.191

In [9]:
res = (2.1914815820078**2 + 1.2652524812963**2 + 10.162337378619**2)**0.5
res

10.472657559692419

In [10]:
plot_kkr(struc0, silent= True, canvas=(1600,1600), repeat_uc= (1,1,1))

structure has vacancies, need to remove empty sites for plotting
removing atom kind name 'HX' @ 1.8747016921119e-16,2.5305049625927,23.373375970824
removing atom kind name 'HX' @ -4.6526674632394e-17,1.2427329655867e-16,27.438310922272
removing atom kind name 'HX' @ 1.7975343758741e-16,2.5305049625927,29.470778397996
removing atom kind name 'HX' @ -3.8809943008617e-17,1.0071414359274e-16,21.3409084951
removing atom kind name 'HX' @ 2.1914815820078,1.2652524812963,25.405843446548


ValueError: top_level must have an `_ipython_display_` method

In [10]:
from pymatgen.core import Lattice, Structure, Molecule
## Here a_new = a-c, b_new= b-c, c_new = c
## change in basis as l_1*a-c, l_2*b-c, l_2c for each atom.
lattice = Lattice(BRAVAIS_arr_new)

original_coors = [[0.0000, 0.0000, 0.0000],
         [1.2120, 1.2120, 0.2120], # [l_1, l_2, l_3]
         [0.7880, 0.7880, 0.7880],
         [1.4000, 0.4000, 0.4000],
         [1.6000, 0.6000, 0.6000],
        ]
eddited_coors = [[0.0000, 0.0000, 0.0000],
         [1.2120, 1.2120, 0.2120 + 1.2120 + 1.2120],
         [0.7880, 0.7880, 0.7880 + 0.7880 + 0.7880],
         [1.4000, 0.4000, 0.4000 + 1.4000 + 0.4000],
         [1.6000, 0.6000, 0.6000 + 1.6000 + 0.6000],
        ]

specis = ['Te', 'Te', 'Te', 'Bi', 'Bi']

struc_new = Structure(lattice, species= specis, coords= eddited_coors)

struc_new_aiida = StructureData()

struc_new_aiida.set_pymatgen_structure(struc_new)
#print(struc_new_aiida.store()) 
## delete node 105516, 105517,  105518, 105519, 105520, 105521,105857, 105858, 105859

struc_new_aiida= load_node(105859)
plot_kkr(struc_new_aiida, struc_plot= True, silent= True, repeat_uc= (1,1,1))
print(struc_new_aiida.pk)

GridspecLayout(children=(Renderer(camera=PerspectiveCamera(fov=10.0, position=(0.0, 0.0, 183.56475962204297), …

105859


In [214]:
from pymatgen.core import Lattice, Structure, Molecule
## Here a_new = a-c, b_new= b-c, c_new = c
## No change in basis
lattice = Lattice(BRAVAIS_arr_new)

coors = [[0.0000, 0.0000, 0.0000],
         [1.2120, 1.2120, 0.2120],
         [0.7880, 0.7880, 0.7880],
         [1.4000, 0.4000, 0.4000],
         [1.6000, 0.6000, 0.6000],
        ]

specis = ['Te', 'Te', 'Te', 'Bi', 'Bi']

struc_new = Structure(lattice, species= specis, coords= coors)

struc_new_aiida = StructureData()

struc_new_aiida.set_pymatgen_structure(struc_new)
#print(struc_new_aiida.store()) 
## delete node 105516, 105517,  105518, 105519, 105520, 105521,105857 
struc_new_aiida= load_node(105857)
plot_kkr(struc_new_aiida, strucplot= True, silent= True, canvas=(400,400), repeat_uc= (1,1,1), label= ['k','k','k','k','k'])
print(struc_new_aiida.pk)
struc_new_aiida.get_pymatgen()

GridspecLayout(children=(Renderer(camera=PerspectiveCamera(fov=10.0, position=(0.0, 0.0, 81.26375056501648), p…

105857


Structure Summary
Lattice
    abc : 4.38296316401566 4.38296316401566 10.472657559692438
 angles : 102.07884028880835 102.07884028880835 59.99999999999911
 volume : 169.06740315520517
      A : 2.1914815820078 3.795757443889 0.0
      B : -2.1914815820078 3.795757443889 0.0
      C : -8.1101110346236e-17 -2.5305049625927 10.162337378619
PeriodicSite: Te (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Te (-0.0000, 8.6644, 2.1544) [1.2120, 1.2120, 0.2120]
PeriodicSite: Te (-0.0000, 3.9881, 8.0079) [0.7880, 0.7880, 0.7880]
PeriodicSite: Bi (2.1915, 5.8202, 4.0649) [1.4000, 0.4000, 0.4000]
PeriodicSite: Bi (2.1915, 6.8324, 6.0974) [1.6000, 0.6000, 0.6000]

In [223]:
from pymatgen.core import Lattice, Structure, Molecule

lattice = Lattice(BRAVAIS_arr_new)
## Here a_new = a-c, b_new= b-c, c_new = c
## change in basis so that looks beautyful
coors = [[0.0000, 0.0000, 0.0000],
         [1.2120, 1.2120, 1.1020],
         [0.7880, 0.7880, 0.50],
         [0.9000, 0.4000, 0.2500],
         [0.6000, 0.6000, 0.8700],
        ]

specis = ['Te', 'Te', 'Te', 'Bi', 'Bi']

struc_new = Structure(lattice, species= specis, coords= coors)

#struc_new_aiida = StructureData()

#struc_new_aiida.set_pymatgen_structure(struc_new)
#print(struc_new_aiida.store()) ## delete node 105516, 105517,  105518, 105519, 105520, 105521, 

#struc_new_aiida 105536
struc_new_aiida = load_node(105536)
plot_kkr(struc_new_aiida, silent= True, canvas=(400,400), repeat_uc= (1,1,1), noshow= True)


GridspecLayout(children=(Renderer(camera=PerspectiveCamera(fov=10.0, position=(0.0, 0.0, 93.02820337663525), p…

In [49]:
with open(struc0.cif, 'w') as _f:
    _f.writelines(struc.get_cif().get_content())

AttributeError: 'StructureData' object has no attribute 'cif'

In [22]:
from pymatgen.core import Lattice, Structure, Molecule

coords = [[0, 0, 0], [0.75,0.5,0.75]]
lattice = Lattice.from_parameters(a=3.84, b=3.84, c=3.84, alpha=120,
                                  beta=90, gamma=60)
struct = Structure(lattice, ["Si", "Si"], coords)
struct.get_space_group_info

<bound method IStructure.get_space_group_info of Structure Summary
Lattice
    abc : 3.84 3.8399999999999994 3.84
 angles : 119.99999999999999 90.0 60.00000000000001
 volume : 40.038580810231124
      A : 3.84 0.0 2.351321854362918e-16
      B : 1.92 2.7152900397563426 -1.919999999999999
      C : 0.0 0.0 3.84
PeriodicSite: Si (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: Si (3.8400, 1.3576, 1.9200) [0.7500, 0.5000, 0.7500]>

In [24]:
plot_kkr(struc0, silent= True, canvas_size= (400,400))

structure has vacancies, need to remove empty sites for plotting
removing atom kind name 'HX' @ 1.8747016921119e-16,2.5305049625927,23.373375970824
removing atom kind name 'HX' @ -4.6526674632394e-17,1.2427329655867e-16,27.438310922272
removing atom kind name 'HX' @ 1.7975343758741e-16,2.5305049625927,29.470778397996
removing atom kind name 'HX' @ -3.8809943008617e-17,1.0071414359274e-16,21.3409084951
removing atom kind name 'HX' @ 2.1914815820078,1.2652524812963,25.405843446548


GridspecLayout(children=(Renderer(camera=PerspectiveCamera(fov=10.0, position=(0.0, 0.0, 202.64102677652383), …

In [194]:
pym = struc0.get_pymatgen()
#pym.get_primitive_structure
struc0

<StructureData: uuid: c1626804-5b71-450c-91c4-aa7197d85279 (pk: 234)>

In [19]:
#with open('strucname.cif', 'w') as _f:
  #  _f.writelines(struc0.get_cif().get_content())

In [166]:
# plot_kkr(234, strucplot= True, silent= True)
struc = load_node(234)
struc.cell

[[2.1914815820078, 1.2652524812963, 10.162337378619],
 [-2.1914815820078, 1.2652524812963, 10.162337378619],
 [-8.1101110346236e-17, -2.5305049625927, 10.162337378619]]

In [169]:
r = (2.1914815820078**2 + 1.2652524812963**2)**0.5
r

2.5305049625926155

In [14]:
li1= [2.1914815820078, 1.2652524812963, 10.162337378619]
alat= np.sqrt(np.sum([i**2 for i in li1]))
alat*(3)**(1/3)
15.10*1.6

24.16

In [15]:
from pymatgen.core import Lattice, Structure, Molecule
original_cell = [
                 [2.1914815820078, 1.2652524812963, 10.162337378619],
                 [-2.1914815820078, 1.2652524812963, 10.162337378619],
                 [-8.1101110346236e-17, -2.5305049625927, 10.162337378619]
                ]

z= original_cell[2]

for i in range(len(original_cell)-1):
    original_cell[i]= list(np.array(original_cell[i])- np.array(z))

original_cell

latice = Lattice(original_cell)

atom_coordinates = [[0.0, 0.0, 0.0],
                   [0.30, 0.50, 0.2],
                   [0.30, 0.25, 0.45],
                   [0.60, 0.50, 0.65],
                   [0.30, 0.30, 0.8]]
struc = Structure(latice, ['Te', 'Bi', 'Te', 'Bi', 'Te'], atom_coordinates)
original_cell

[[2.1914815820078, 3.795757443889, 0.0],
 [-2.1914815820078, 3.795757443889, 0.0],
 [-8.1101110346236e-17, -2.5305049625927, 10.162337378619]]

In [17]:
#struc

In [18]:
.3*-1.919999999999999 + 0.9000*3.84


2.8800000000000003

In [54]:
#To colloect the Jij Data
accending_distance_group = [104,102, 118, 115 ]

In [8]:
ll = {'a':10}
hasattr(ll,'a')

False

In [9]:
ll.__at

TypeError: expected 1 argument, got 0